In [46]:
import numpy as np
import cPickle, gzip

from neon import logger as neon_logger
from neon.callbacks.callbacks import Callbacks
from neon.data import MNIST
from neon.initializers import Gaussian, GlorotUniform
from neon.layers import Affine, BranchNode, Multicost, Tree, GeneralizedCost
from neon.models import Model
from neon.optimizers import RMSProp
from neon.transforms import Rectlin, Logistic, Softmax
from neon.transforms import CrossEntropyMulti, Misclassification
from neon.util.argparser import NeonArgparser
from neon.data import ArrayIterator
from neon.backends import gen_backend

In [2]:
# Set up a number of initial variables for use with baseline
NUM_TASKS = 3; # number of learning tasks (for multi-task learning)
NUM_FOLDS = 10; # number of folds for training (main cross validation loop)
NUM_EPOCH = 5; # number of epochs



truth_a_arr = []
pred_a_arr = []

truth_b_arr = []
pred_b_arr = []

truth_c_arr = []
pred_c_arr = []


In [3]:
for fold in range( NUM_FOLDS ):

    features_train = []
    labels_train = []
    truths_train = []

    features_test = []
    labels_test = []
    truths_test = []

    n_out = []

    for task in range( NUM_TASKS ):
        file_post = '.' + str(task) + '.' + str(fold) + '.pkl.gz'
        fname_train = 'train/train' + file_post; 
        fname_test  = 'test/test' + file_post; 

        with gzip.open( fname_train, 'rb' ) as f:
            feature_train, label_train = cPickle.load( f )

        with gzip.open( fname_test, 'rb') as f:
            feature_test, label_test = cPickle.load( f )

        features_train.append( feature_train )
        labels_train.append( label_train )

        features_test.append( feature_test )
        labels_test.append( label_test )

        mv = numpy.max( label_train )
        truth_train = numpy.zeros( ( len( label_train ), mv + 1 ) )
        for i in range( len( label_train ) ):
            truth_train[ i, label_train[ i ] ] = 1

        truths_train.append( truth_train )

        mv = numpy.max( label_test )
        truth_test = numpy.zeros( ( len( label_test ), mv + 1 ) )
        for i in range( len( label_test ) ):
            truth_test[ i, label_test[ i ] ] = 1

        truths_test.append( truth_test )

        n_out.append( mv + 1 )

    flen = len( feature_train[ 0 ] ); # input feature length is set to 400 for now based on the training examples available.


In [50]:
n_out = np.array(n_out)

In [48]:
from neon.backends import gen_backend
be = gen_backend(backend='cpu',batch_size=10)

In [53]:
train_t1 = ArrayIterator(X=features_train[0], y=labels_train[0], nclass=n_out[0])
train_t2 = ArrayIterator(X=features_train[1], y=labels_train[1], nclass=n_out[1])
train_t3 = ArrayIterator(X=features_train[2], y=labels_train[2], nclass=n_out[2])

test_t1 = ArrayIterator(X=features_test[0], y=labels_test[0], nclass=n_out[0])
test_t2 = ArrayIterator(X=features_test[1], y=labels_test[1], nclass=n_out[1])
test_t3 = ArrayIterator(X=features_test[2], y=labels_test[2], nclass=n_out[2])

In [7]:
b1 = BranchNode(name='b1')
b2 = BranchNode(name='b2')
init = GlorotUniform()

In [8]:
p1 = [Affine(nout=flen, name="t1_input", activation=Rectlin(), init=init), 
      b1,
      Affine(nout=flen, name="share", activation=Rectlin(), init=init),
      b2,
      Affine(nout=flen, name="t1_3", activation=Rectlin(), init=init),
      Affine(nout=256, name="t1_4", activation=Rectlin(), init=init),
      Affine(nout=n_out[0], name="t2_out", activation=Rectlin(), init=init)]
p2 = [Affine(nout=flen, name="t2_input", activation=Rectlin(), init=init),
     b1,
     Affine(nout=flen, name="share", activation=Rectlin(), init=init),
     b2, 
      Affine(nout=flen, name="t2_3", activation=Rectlin(), init=init),
      Affine(nout=256, name="t2_4", activation=Rectlin(), init=init),
      Affine(nout=n_out[1], name="t2_out", activation=Rectlin(), init=init)]
p3 = [Affine(nout=flen, name="t3_input", activation=Rectlin(), init=init),
     b1,
     Affine(nout=flen, name="share", activation=Rectlin(), init=init),
     b2, 
      Affine(nout=flen, name="t3_3", activation=Rectlin(), init=init),
      Affine(nout=256, name="t3_4", activation=Rectlin(), init=init),
      Affine(nout=n_out[2], name="t3_out", activation=Rectlin(), init=init)]

In [75]:
model_1 = Model(layers=p1)
model_2 = Model(layers=p2)
model_3 = Model(layers=p3)

In [76]:
optimizer = RMSProp(learning_rate=0.001)
cost = GeneralizedCost(costfunc=CrossEntropyMulti())
                        
callbacks_1 = Callbacks(model_1, eval_set=test_t1,
                      eval_freq=NUM_EPOCH)
callbacks_2 = Callbacks(model_2, eval_set=test_t2,
                      eval_freq=NUM_EPOCH)
callbacks_3 = Callbacks(model_3, eval_set=test_t3,
                      eval_freq=NUM_EPOCH)

In [77]:
model_1.fit(train_t1, optimizer=optimizer, 
            cost=cost, 
            callbacks=callbacks_1, num_epochs=NUM_EPOCH)
model_2.fit(train_t2, optimizer=optimizer, cost=cost, 
            callbacks=callbacks_2, num_epochs=NUM_EPOCH)
model_3.fit(train_t3, optimizer=optimizer, cost=cost, 
            callbacks=callbacks_3, num_epochs=NUM_EPOCH)

Epoch 0   [Train |████████████████████|  120/120  batches, 0.01 cost, 1.13s]
Epoch 1   [Train |████████████████████|  120/120  batches, 0.04 cost, 1.23s]
Epoch 2   [Train |████████████████████|  120/120  batches, 0.06 cost, 1.23s]
Epoch 3   [Train |████████████████████|  120/120  batches, 0.04 cost, 1.13s]
Epoch 4   [Train |████████████████████|  120/120  batches, 0.03 cost, 1.09s] [CrossEntropyMulti Loss 1.44, 0.02s]
Epoch 0   [Train |████████████████████|  120/120  batches, 0.01 cost, 1.13s]
Epoch 1   [Train |████████████████████|  120/120  batches, 0.00 cost, 1.34s]
Epoch 2   [Train |████████████████████|  120/120  batches, 0.01 cost, 1.37s]
Epoch 3   [Train |████████████████████|  120/120  batches, 0.01 cost, 1.31s]
Epoch 4   [Train |████████████████████|  120/120  batches, 0.02 cost, 1.31s] [CrossEntropyMulti Loss 25.95, 0.01s]
Epoch 0   [Train |████████████████████|  120/120  batches, 50.00 cost, 1.27s]
Epoch 1   [Train |████████████████████|  120/120  batches, 50.00 cost, 1.38s]